# Kubernetes demo

# Description
In this demo I'll show you how to create a Postgres cluster with CloudNativePG kubernetes operator. The features that I want to show you are:
- Kubernetes plugin install
- CloudNativePG operator install
- Postgres cluster install
- Insert data in the cluster
- Switchover (promote)
- Failover
- Backup
- Recovery
- Rolling updates (minor and major)

# Prerequisites
- K8s environment
- Tested with Docker Desktop with Kubernetes on Mac 


## Demo prep
### First we are going to start Minio

In [ ]:
ls

In [ ]:
kubectl create secret generic minio-creds \
  --from-literal=MINIO_ACCESS_KEY=admin \
  --from-literal=MINIO_SECRET_KEY=password

### Then we are going to run the Minio container

In [ ]:
docker run -p 9000:9000 -p 9001:9001 \
           -e MINIO_ROOT_USER=admin \
           -e MINIO_ROOT_PASSWORD=password \
           -d --rm \
           minio/minio server /data \
           --console-address ":9001"

### Open the Minio console

In [ ]:
open http://localhost:9001

## Configuration

## Install plugin

In [ ]:
curl -sSfL https://github.com/cloudnative-pg/cloudnative-pg/raw/main/hack/install-cnpg-plugin.sh | sh -s -- -b /usr/local/bin

## Install operator

In [ ]:
version1=`kubectl-cnpg version | awk '{ print $2 }' | awk -F":" '{ print $2}'`
version2=${version1%??}
kubectl apply -f https://raw.githubusercontent.com/cloudnative-pg/cloudnative-pg/release-$version2/releases/cnpg-$version1.yaml

## Check operator installed

In [ ]:
kubectl get deploy -n cnpg-system cnpg-controller-manager -o wide

# Create cluster

## Install cluster

In [ ]:
kubectl apply -f cluster-example.yaml

## Show status

`watch -c -n 1 kubectl-cnpg status cluster-example`

## Insert data

In [ ]:
kubectl exec -i cluster-example-1 -- psql < sqltest.sql

## Check data

In [ ]:
kubectl exec -it cluster-example-1 -- psql -c "select * from test limit 15;"

In [ ]:
kubectl exec -it cluster-example-1 -- psql -c "select count(*) from test;"

# Promotion 
## Promotion of node 2 to Primary

In [ ]:
kubectl-cnpg promote cluster-example cluster-example-2

# In-place rolling upgrade minor version
## Update from Postgres 14.2 to 14.5

In [ ]:
kubectl apply -f cluster-example-upgrade.yaml

# Backup & restore
## Backup cluster

In [ ]:
kubectl apply -f backup.yaml

## Verify backup

In [ ]:
kubectl describe backup backup-test

## Restore to new cluster

In [ ]:
kubectl apply -f restore.yaml

## Verify restore

`watch -c -n 1 kubectl-cnpg status cluster-restore`

In [ ]:
kubectl exec -it cluster-restore-1 -- psql -c "select * from test limit 15;"

In [ ]:
kubectl exec -it cluster-restore-1 -- psql -c "select count(*) from test;"

# Failover 
## Delete Primary pod

In [ ]:
kubectl delete pod cluster-example-1 --force

`watch -c -n 1 kubectl-cnpg status cluster-example`

# Migration major version
## Create Postgres 13 cluster

In [ ]:
kubectl apply -f cluster-example-13.yaml

`watch -c -n 1 kubectl-cnpg status cluster-example-13`

## Add data to cluster

In [ ]:
kubectl exec -it cluster-example-13-1 -- psql app < sqltest.sql

## Verify inserted data

In [ ]:
kubectl exec -it cluster-example-13-1 -- psql app -c "select * from test limit 15;"

In [ ]:
kubectl exec -it cluster-example-13-1 -- psql app -c "select count(*) from test;" app

## Create new Postgres 15 cluster based on Postgres 13 cluster

In [ ]:
kubectl apply -f cluster-example-upgrade-13-to-15.yaml

## Verify data in new cluster

In [ ]:
kubectl exec -it cluster-example-15-1 -- psql app -c "select * from test limit 15;"

In [ ]:
kubectl exec -it cluster-example-15-1 -- psql app -c "select count(*) from test;"

# End of demo

## How to deploy and access the Kubernetes Dashboard

In [ ]:
./dashboard_install.sh

# Links
[https://github.com/minio/minio](https://github.com/minio/minio)

[https://cloudnative-pg.io](https://cloudnative-pg.io)

# Cleanup

In [ ]:
kubectl delete backup backup-test
kubectl delete deployments.apps -n cnpg-system cnpg-controller-manager
version1=`kubectl-cnpg version | awk '{ print $2 }' | awk -F":" '{ print $2}'`
version2=${version%??}
kubectl delete -f https://raw.githubusercontent.com/cloudnative-pg/cloudnative-pg/release-$version2/releases/cnpg-$version1.yaml
kubectl delete secret minio-creds
docker ps | grep minio | awk '{print $1}' | xargs -I % docker stop %
kubectl delete cluster cluster-example-13
kubectl delete cluster cluster-example-15
kubectl delete cluster cluster-example
kubectl delete cluster cluster-restore
docker pull ghcr.io/cloudnative-pg/postgresql:13
docker pull ghcr.io/cloudnative-pg/postgresql:14.2
docker pull ghcr.io/cloudnative-pg/postgresql:14.5
docker pull ghcr.io/cloudnative-pg/postgresql:15.1

## Uninstall Kubernetes Dashboard

In [ ]:
./dashboard_uninstall.sh